<a href="https://colab.research.google.com/github/sowmini4901/SceneSegmentation/blob/main/SceneSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install focal-loss

In [ ]:
pip install tensorflow

In [ ]:
#importing necessary libraries for project
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import os
import pickle
import torch
from sklearn import preprocessing
from focal_loss import BinaryFocalLoss

In [ ]:
#mounting google drive for accessing files in it
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#path for dataset
folder='/content/gdrive/MyDrive/movies/data'
files=os.listdir(folder)
noOfFiles = len(files) 
shotsforMovie = np.zeros((noOfFiles), dtype = np.uint16)



In [ ]:
#initializing the variables needed
k=0
j=0
counter=0


In [ ]:
#going through each movie file
for i in files:
  filename=folder+'/'+i
  with open(filename,'rb') as f:
     data=pickle.load(f)
  counter=counter+1
  if counter==3:
    break
  #converting tensors to arrays for features
  place=data['place']
  place=place.data.numpy()
  placeSize=place.shape[1]

  cast=data['cast']
  cast=cast.data.numpy()
  castSize=cast.shape[1]

  action=data['action']
  action=action.data.numpy()
  actionSize=action.shape[1]

  audio=data['audio']
  audio=audio.data.numpy()
  audioSize=audio.shape[1]

  #combining all the features into a huge table of 1101 rows(no.of shots) and 3584 columns(2048+512+512+512)
  x = np.hstack((place, cast, action, audio))
  y = data['scene_transition_boundary_ground_truth']
 
  N = x.shape[0] 
  x_fold = np.zeros((N - 1, 2*x.shape[1])) 

  #adding the adjoining shots to abstract features from it.
  for i in range(N - 1):
    x_fold[i,:] = np.hstack((x[i,:], x[i+1,:])) 
    
  if (j == 0):
    X = x_fold
    Y = y

  else:
    X = np.concatenate((X, x_fold), axis = 0)
    Y = np.concatenate((Y, y))
    

    
  j = j + 1
  k = k + 1

print(X.shape)
print(Y.shape)

#converting the predictions to 0 and 1 from false and true
M = Y.shape[0]
YTruth = np.zeros((M), dtype = np.uint8)
for i in range(M):
  if(Y[i] == True):
      YTruth[i] = 0
  elif(Y[i] == False):
      YTruth[i] = 1

# Create train and test datasets for cross-validation
shots_in_dataset = np.zeros((noOfFiles), dtype = np.uint32)
previous_shots = 0
for i in range(noOfFiles):
  shotsforMovie[i] = shotsforMovie[i] - 1
  shots_in_dataset[i] = shotsforMovie[i] + previous_shots
  previous_shots = shots_in_dataset[i]
  
#print(shots_per_movie)
#print(shots_in_dataset)

iter_sizes = np.zeros((4), dtype = np.uint32)
iter_sizes[0] = 0.2*noOfFiles - 1
iter_sizes[1] = 0.4*noOfFiles - 1
iter_sizes[2] = 0.6*noOfFiles - 1
iter_sizes[3] = 0.8*noOfFiles - 1
  
# Create IDs for cross-validation
iter_ids = np.zeros((5,2), dtype = np.uint32)
iter_ids[0,0] = 0 
iter_ids[0,1] = shots_in_dataset[iter_sizes[0]]

iter_ids[1,0] = shots_in_dataset[iter_sizes[0]]
iter_ids[1,1] = shots_in_dataset[iter_sizes[1]]

iter_ids[2,0] = shots_in_dataset[iter_sizes[1]]
iter_ids[2,1] = shots_in_dataset[iter_sizes[2]]

iter_ids[3,0] = shots_in_dataset[iter_sizes[2]]
iter_ids[3,1] = shots_in_dataset[iter_sizes[3]]

iter_ids[4,0] = shots_in_dataset[iter_sizes[3]]
iter_ids[4,1] = M

array_dims = np.zeros((8), dtype = np.int32)
array_dims[0] = placeSize
array_dims[1] = castSize #feat1_size + feat2_size
array_dims[2] = actionSize #feat1_size + feat2_size + feat3_size
array_dims[3] = audioSize #feat1_size + feat2_size + feat3_size + feat4_size
array_dims[4] = array_dims[0] #array_dims[3] + feat1_size
array_dims[5] = array_dims[1] #array_dims[3] + feat1_size + feat2_size
array_dims[6] = array_dims[2] #array_dims[3] + feat1_size + feat2_size + feat3_size
array_dims[7] = array_dims[3] #a

inputs = tf.keras.layers.Input(shape = (X.shape[1]))
x1, x2, x3, x4, x5, x6, x7, x8 = tf.split(inputs, array_dims, axis = 1)

x1_r = tf.keras.layers.Reshape((placeSize, 1))(x1)
#x1_b = tf.keras.layers.BatchNormalization(axis=1)(x1_r)

x5_r = tf.keras.layers.Reshape((placeSize, 1))(x5)
#x5_b = tf.keras.layers.BatchNormalization(axis=1)(x5_r)

#x2b = tf.keras.layers.BatchNormalization(axis=1)(x2)
x2_conc = tf.keras.layers.concatenate([x2, x2, x2, x2], axis = 1) # create vectors of size (None, 2048) from size (None, 512)
x2_r = tf.keras.layers.Reshape((placeSize, 1))(x2_conc)
print(x2_r.shape)

#x3b = tf.keras.layers.BatchNormalization(axis=1)(x3)
x3_conc = tf.keras.layers.concatenate([x3, x3, x3, x3], axis = 1)
x3_r = tf.keras.layers.Reshape((placeSize, 1))(x3_conc)
print(x3_r.shape)

#x4b = tf.keras.layers.BatchNormalization(axis=1)(x4)
x4_conc = tf.keras.layers.concatenate([x4, x4, x4, x4], axis = 1)
x4_r = tf.keras.layers.Reshape((placeSize, 1))(x4_conc)
print(x4_r.shape)

#x6b = tf.keras.layers.BatchNormalization(axis=1)(x6)
x6_conc = tf.keras.layers.concatenate([x6, x6, x6, x6], axis = 1) # create vectors of size (None, 2048) from size (None, 512)
x6_r = tf.keras.layers.Reshape((placeSize, 1))(x6_conc)
print(x6_r.shape)

#x7b = tf.keras.layers.BatchNormalization(axis=1)(x7)
x7_conc = tf.keras.layers.concatenate([x7, x7, x7, x7], axis = 1)
x7_r = tf.keras.layers.Reshape((placeSize, 1))(x7_conc)
print(x7_r.shape)

#x8b = tf.keras.layers.BatchNormalization(axis=1)(x8)
x8_conc = tf.keras.layers.concatenate([x8, x8, x8, x8], axis = 1)
x8_r = tf.keras.layers.Reshape((placeSize, 1))(x8_conc)
print(x8_r.shape)

shot_1 = tf.keras.layers.concatenate([x1_r, x2_r, x3_r, x4_r], axis = 2) # create a vector of size (None, 2048, 4)
print(shot_1.shape)

shot_2 = tf.keras.layers.concatenate([x5_r, x6_r, x7_r, x8_r], axis = 2) # create a vector of size (None, 2048, 4)
print(shot_2.shape)

# building VGG net(standard architecture)
conv1 = tf.keras.Sequential()
conv1.add(tf.keras.layers.Reshape((1, placeSize, 4)))

conv1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 2048, 4)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 2048, 64)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2)))  #input size(none,1,2048,64)


conv1.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 1024, 64)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 1024,64)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2)))   #input size(none,1,512,128)


conv1.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 256, 256)

conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())


conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 128, 512)

conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())


conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 64, 512)
conv1.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 32, 512)

conv1.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 1), strides=1, padding='same', activation='linear')) #input size (None, 1, 32, 512)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())

conv1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 32, 512)
conv1.add(tf.keras.layers.BatchNormalization(axis=3))
conv1.add(tf.keras.layers.ReLU())#input size (None, 1, 32, 64)



encoded_left = conv1(shot_1)
print(conv1.summary())

conv2 = tf.keras.Sequential()
conv2.add(tf.keras.layers.Reshape((1, placeSize, 4)))

conv2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 2048, 4)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 2048, 64)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2)))  #input size(none,1,2048,64)


conv2.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 1024, 64)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 1024,64)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2)))   #input size(none,1,512,128)


conv2.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 512, 128)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 256, 256)

conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())


conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 256, 256)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 128, 512)

conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())


conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 128, 512)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 64, 512)
conv2.add(tf.keras.layers.MaxPool2D(pool_size=(1,2),strides=(2,2))) #output size (None, 1, 32, 512)

conv2.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 1), strides=1, padding='same', activation='linear')) #input size (None, 1, 32, 512)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())

conv2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same', activation='linear')) #input size (None, 1, 32, 512)
conv2.add(tf.keras.layers.BatchNormalization(axis=3))
conv2.add(tf.keras.layers.ReLU())#input size (None, 1, 32, 64)

encoded_right = conv2(shot_2)
print(conv2.summary())

combine = tf.keras.layers.concatenate([encoded_left, encoded_right], axis = 2)
flat = tf.keras.layers.Flatten()(combine) #output size (None, 65536)
d1 = tf.keras.layers.Dropout(rate=0.2)(flat)
d2 = tf.keras.layers.Dense(4096, activation='relu')(d1)
d2r = tf.keras.layers.Dropout(rate=0.2)(d2)
d3 = tf.keras.layers.Dense(2048, activation='relu')(d2r)
d3r = tf.keras.layers.Dropout(rate=0.2)(d3)
output = tf.keras.layers.Dense(1, activation = 'softmax')(d3r)

model = tf.keras.Model(inputs = inputs, outputs = output)
opt = tf.keras.optimizers.Adam(learning_rate=0.02)

model.compile(optimizer = opt, loss = BinaryFocalLoss(pos_weight = 70, gamma=1.5), metrics=[tf.keras.metrics.Accuracy()])
model.summary()

training_log = 'crossval_fold_' + '.txt'
csv_logger = tf.keras.callbacks.CSVLogger(training_log, append = True, separator=' ')
metrics = model.fit(X, YTruth, epochs=4, validation_split= 0.2, verbose=2, batch_size = 32)

(2237, 7168)
(2237,)
(None, 2048, 1)
(None, 2048, 1)
(None, 2048, 1)
(None, 2048, 1)
(None, 2048, 1)
(None, 2048, 1)
(None, 2048, 4)
(None, 2048, 4)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_8 (Reshape)         (None, 1, 2048, 4)        0         
                                                                 
 conv2d (Conv2D)             (None, 1, 2048, 64)       832       
                                                                 
 batch_normalization (BatchN  (None, 1, 2048, 64)      256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 1, 2048, 64)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 2048, 64)       12352     
                                       